In [1]:
import pandas as pd 
import numpy as np
from gensim.models import Word2Vec as w2v
import matplotlib.pyplot as plt
from numpy.linalg import norm
from numpy import dot
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from math import radians, cos, sin, asin, sqrt, atan2
import joblib
import warnings
warnings.filterwarnings('ignore')

In [11]:
# User details
# Example of long and lat to use in ui : -96.602464, 46.265207 
user_long = -96.611273
user_lat = 46.268488
business_to_find = 'restaurant'

In [3]:
# Business dataframe with embedded categories
meta_df = pd.read_json('data/business_with_embeddings.json')

In [4]:
# Converting user input to embeddings using our pretrained TF-IDF model
def convert_user_input(user_input):
    ps = PorterStemmer()
    sw = stopwords.words('english')  
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    user_tokens = word_tokenize(user_input.lower())
    user_tokens = [ps.stem(token) for token in user_tokens if token.isalnum() and token not in sw]
    user_text = ' '.join(user_tokens)
    user_vector = vectorizer.transform([user_text])
    return user_vector

In [5]:
# Caculating distance of business from the user
def check_business_distance(long1, lat1, long2, lat2):
    long1 = radians(long1)
    lat1 = radians(lat1)
    long2 = radians(long2)
    lat2 = radians(lat2)
    
    dlon = long2 - long1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2

    c = 2 * asin(sqrt(a))

    r = 6371 # Radius of earth in kilometers

    return (c * r)

In [6]:
# Sorting similar businesses by their distance and ratings
def process_related_business(df):
    df['business_distance'] = [check_business_distance(user_long, user_lat, long, lat) for long, lat in zip(df['longitude'], df['latitude'])]
    df = df.sort_values(by=['business_distance', 'avg_rating', 'num_of_reviews'], ascending=[True, False, False])
    return df

In [7]:
# Find most related business as per user search
def find_business(user_input):
    user_vector = convert_user_input(user_input)
    similar_business = []
    for categories in meta_df['category_embeddings']:
        cosine_similarities = cosine_similarity(user_vector, categories)
        similar_business.append(cosine_similarities.max())
    similar_business = np.where(np.array(similar_business)>=0.8)[0]
    similar_business_df = meta_df.iloc[similar_business]
    similar_business_df = process_related_business(similar_business_df)
    return similar_business_df

In [8]:
# Similar business that are nearby and highly rated
df = find_business(business_to_find)
df.head(5)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,category_embeddings,business_distance
10159,Pizza Ranch,"Pizza Ranch, 1110 3rd Ave N, Wahpeton, ND 58075",0x52c94922c9c093e1:0x5c3eba716b832f2f,"Buffet chain offering pizza, fried chicken & c...",46.266396,-96.616728,"[Buffet restaurant, Chicken restaurant, Delive...",3.9,334,$$,"[[Saturday, 11AM–8PM], [Sunday, 11AM–8PM], [Mo...","{'Service options': ['Curbside pickup', 'No-co...",Closed ⋅ Opens 11AM,"[0x52c94920b8f24943:0x5da68b54f67a4194, 0x52c9...",https://www.google.com/maps/place//data=!4m2!3...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.479492
300,Lovin Nutrition,"Lovin Nutrition, 118 6th St N suite b, Wahpeto...",0x52c949b90e9b1943:0x4c744a591e2501e8,None,46.264664,-96.608174,[Restaurant],4.7,4,None,None,"{'Service options': ['Takeout', 'Dine-in', 'De...",None,None,https://www.google.com/maps/place//data=!4m2!3...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.487392
9620,Burger King,"Burger King, 924 Dakota Ave, Wahpeton, ND 58075",0x52c949230ad9ff4b:0x82cab4bb66d54d1b,Well-known fast-food chain serving grilled bur...,46.264418,-96.613845,"[Restaurant, Fast food restaurant]",3.4,357,$,"[[Friday, 6AM–11PM], [Saturday, 6AM–11PM], [Su...","{'Service options': ['Outdoor seating', 'Drive...",Open ⋅ Closes 11PM,"[0x52c9493ce8295ca7:0x414104f68ab9595c, 0x52c9...",https://www.google.com/maps/place//data=!4m2!3...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.493880
10434,Wahpeton Deli,"Wahpeton Deli, 614 Dakota Ave, Wahpeton, ND 58075",0x52c9492141e9cc6f:0xb18619b348770fbd,None,46.264141,-96.609154,"[Deli, Restaurant]",4.5,117,$,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–3PM], ...","{'Service options': ['In-store shopping', 'Tak...",Closed ⋅ Opens 10AM,"[0x52c948db8f5ccd45:0x1c8eb363f4d7c92d, 0x52c9...",https://www.google.com/maps/place//data=!4m2!3...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.510079
10165,Taco John's,"Taco John's, 801 Dakota Ave, Wahpeton, ND 58075",0x52c9492394b087c9:0x9d0e974b9412f2e3,"Fast-food chain featuring tacos, burritos, nac...",46.263649,-96.611600,"[Fast food restaurant, Breakfast restaurant, D...",3.9,238,$,"[[Saturday, 7AM–10PM], [Sunday, Closed], [Mond...","{'Service options': ['Drive-through', 'Takeout...",Closed ⋅ Opens 7AM,"[0x52c949230ad9ff4b:0x82cab4bb66d54d1b, 0x52c9...",https://www.google.com/maps/place//data=!4m2!3...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.538672


In [9]:
df.iloc[0]['url']

'https://www.google.com/maps/place//data=!4m2!3m1!1s0x52c94922c9c093e1:0x5c3eba716b832f2f?authuser=-1&hl=en&gl=us'